In [1]:
import pandas as pd
import numpy as np
import geojson

In [2]:
with open('data/Places/Richard II_labels.geojson') as f:
    data = geojson.load(f)
    f.close()

FileNotFoundError: [Errno 2] No such file or directory: 'data/Places/Richard II_labels.geojson'

In [21]:
gacc = []
n_gacc = []
features = []
for feat in data['features']:
    g = feat['geometry']['coordinates']
    if g not in gacc:
        gacc.append(g)
        geom = geojson.Point(g)
        n = feat['properties']['place name']
        f1 = geojson.Feature(geometry=geom, properties={'name': n})
        features.append(f1)
        
f_col = geojson.FeatureCollection(features)


In [3]:
a = 9
b = 10
print(a + b)

19


In [22]:
h_plays = ['Titus Andronicus', 'Coriolanus', 'Troilus and Cressida', 'Pericles', 'Timon of Athens', 'Antony and Cleopatra', 'Julius Caesar', 'Henry IV, Part I', 'Henry IV, Part II', 'Henry V', 'Henry VI, Part 1','Henry VI, Part 2', 'Henry VI, Part 3', 'Henry VIII', 'King John', 'Richard II', 'Richard III']

for p in h_plays:
    with open('data/Labels/' + p + '_labels.geojson', 'r') as f:
        data = geojson.load(f)
        f.close()
    gacc = []
    features = []
    for feat in data['features']:
        g = feat['geometry']['coordinates']
        if g not in gacc:
            gacc.append(g)
            geom = geojson.Point(g)
            n = feat['properties']['place name']
            f1 = geojson.Feature(geometry=geom, properties={'name': n})
            features.append(f1)
    f_col = geojson.FeatureCollection(features)
    with open('data/Places/' + p + '_places.geojson', 'w') as f:
        geojson.dump(f_col, f)

In [ ]:
#function to combine these two formats of df and spit out GeoJSON
def to_GeoJSON(text_df, geo_df, gtype):
    if gtype == 'Polygon':
        z = geojson.Polygon
    elif gtype == 'Point':
        z = geojson.Point
    features = []
    for i, row in geo_df.iterrows():
        place = row['Place_Name']
        m_name = row['Modern_Name']
        #df of rows that contain place name
        sub_txt_df = find_rows(text_df, place)
        #add column of just line numbers
        sub_txt_df['Line'] = [get_ln_nums(x, place) for x in sub_txt_df.Dialogue]
        geometry = row.Geometry
        for ix, rowx in sub_txt_df.iterrows():
            props = dict()
            geom = z(geometry)
            props['play'] = rowx.Play
            props['place name'] = place
            props['character'] = rowx['Speaker']
            props['a.s.l.'] = rowx['Line']
            props['sequence'] = rowx['Sequence']
            f1 = geojson.Feature(geometry=geom, properties=props)
            if f1.is_valid == False:
                print(f1.properties['place name'])
                raise Exception('The coordinates for this place are in an invalid format.')
            features.append(f1)
    f_col = geojson.FeatureCollection(features)  
    return f_col
     
geo_data = to_GeoJSON(strict_df, coordinate_df, 'Polygon')